In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import seaborn as sns

from os.path import join

plt.style.use(["seaborn", "thesis"])

/home/jo/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
plt.rc("figure", figsize=(8,4))

# Dataset

In [3]:
from SCFInitialGuess.utilities.dataset import extract_triu_batch, AbstractDataset
from sklearn.model_selection import train_test_split

# fetch dataset
data_path = "../../../dataset/TSmall_sto3g/"
postfix = "TSmall_sto3g"
dim = 26
N_ELECTRONS = 30
#data_path = "../butadien/data/"
#postfix = ""
#dim = 26


def split(x, y, ind):
    return x[:ind], y[:ind], x[ind:], y[ind:]

S = np.load(join(data_path, "S" + postfix + ".npy"))
P = np.load(join(data_path, "P" + postfix + ".npy"))
#F = np.load(join(data_path, "F" + postfix + ".npy"))

molecules = np.load(join(data_path, "molecules" + postfix + ".npy"))



ind = int(0.8 * len(molecules))
molecules = (molecules[:ind], molecules[ind:])

s_triu = extract_triu_batch(S, dim)
p_triu = extract_triu_batch(P, dim)

s_train, p_train, s_test, p_test = split(s_triu, p_triu, ind)

# Fetching Descriptor 

In [4]:
import pickle

descriptor = pickle.load(open("../../../models/ButadienTDescriptor/descriptor.dump", "rb"))

In [22]:
mu, std = np.load("../../../models/ButadienTDescriptor/normalisation.npy")

In [20]:
from SCFInitialGuess.utilities.dataset import StaticDataset

dataset = StaticDataset(
    train=(s_train, p_train),
    validation=(None, None),
    test=(s_test, p_test),
    mu=0,
    std=1
)

In [7]:
filepath = "../../../models/ButadienTDescriptor/model_descriptos_" + postfix + ".h5"
model = keras.models.load_model(filepath)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 700)               1386700   
_________________________________________________________________
dense_1 (Dense)              (None, 500)               350500    
_________________________________________________________________
dense_2 (Dense)              (None, 400)               200400    
_________________________________________________________________
dense_3 (Dense)              (None, 351)               140751    
Total params: 2,078,351
Trainable params: 2,078,351
Non-trainable params: 0
_________________________________________________________________


# Make Guess 

In [8]:
from SCFInitialGuess.utilities.dataset import AbstractDataset

G = []
for mol in molecules[1]:
    G.append(
        descriptor.calculate_all_descriptors(mol).flatten()
    )

G = np.asarray(G)
G_norm = AbstractDataset.normalize(G, mean=mu, std=std)[0]
G_norm.shape

(201, 1980)

In [9]:
p_nn = model.predict(G_norm)
p_nn.shape

(201, 351)

# Ananlysis 

In [16]:
from SCFInitialGuess.utilities.usermessages import Messenger as msg
msg.print_level = 0

70551.0

In [21]:
from SCFInitialGuess.utilities.analysis import mf_initializer as mf_initializer
from SCFInitialGuess.utilities.analysis import make_results_str, measure_all_quantities

print(make_results_str(measure_all_quantities(
    p_nn,
    dataset,
    molecules[1],
    N_ELECTRONS,
    mf_initializer,
    dim,
    is_triu=True,
    is_dataset_triu=True
)))

[ ] 2018-12-10 01:06:41: Iteration calculation: 0
[ ] 2018-12-10 01:06:41: Iteration calculation: 1
[ ] 2018-12-10 01:06:41: Iteration calculation: 2
[ ] 2018-12-10 01:06:42: Iteration calculation: 3
[ ] 2018-12-10 01:06:42: Iteration calculation: 4
[ ] 2018-12-10 01:06:43: Iteration calculation: 5
[ ] 2018-12-10 01:06:43: Iteration calculation: 6
[ ] 2018-12-10 01:06:43: Iteration calculation: 7
[ ] 2018-12-10 01:06:44: Iteration calculation: 8
[ ] 2018-12-10 01:06:44: Iteration calculation: 9
[ ] 2018-12-10 01:06:44: Iteration calculation: 10
[ ] 2018-12-10 01:06:45: Iteration calculation: 11
[ ] 2018-12-10 01:06:45: Iteration calculation: 12
[ ] 2018-12-10 01:06:45: Iteration calculation: 13
[ ] 2018-12-10 01:06:46: Iteration calculation: 14
[ ] 2018-12-10 01:06:46: Iteration calculation: 15
[ ] 2018-12-10 01:06:47: Iteration calculation: 16
[ ] 2018-12-10 01:06:47: Iteration calculation: 17
[ ] 2018-12-10 01:06:47: Iteration calculation: 18
[ ] 2018-12-10 01:06:48: Iteration calcul

[ ] 2018-12-10 01:07:51: Iteration calculation: 160
[ ] 2018-12-10 01:07:52: Iteration calculation: 161
[ ] 2018-12-10 01:07:52: Iteration calculation: 162
[ ] 2018-12-10 01:07:53: Iteration calculation: 163
[ ] 2018-12-10 01:07:53: Iteration calculation: 164
[ ] 2018-12-10 01:07:54: Iteration calculation: 165
[ ] 2018-12-10 01:07:54: Iteration calculation: 166
[ ] 2018-12-10 01:07:55: Iteration calculation: 167
[ ] 2018-12-10 01:07:55: Iteration calculation: 168
[ ] 2018-12-10 01:07:56: Iteration calculation: 169
[ ] 2018-12-10 01:07:56: Iteration calculation: 170
[ ] 2018-12-10 01:07:56: Iteration calculation: 171
[ ] 2018-12-10 01:07:57: Iteration calculation: 172
[ ] 2018-12-10 01:07:57: Iteration calculation: 173
[ ] 2018-12-10 01:07:58: Iteration calculation: 174
[ ] 2018-12-10 01:07:58: Iteration calculation: 175
[ ] 2018-12-10 01:07:59: Iteration calculation: 176
[ ] 2018-12-10 01:07:59: Iteration calculation: 177
[ ] 2018-12-10 01:07:59: Iteration calculation: 178
[ ] 2018-12-

## Damping and DIIS 

In [ ]:
def format_results(result):
    if isinstance(result, list):
        out = list(map(
            lambda x: "{:0.5E} +- {:0.5E}".format(*x),
            result
        ))
        out = "\n".join(out)
    else:
        out =  "{:0.5E} +- {:0.5E}".format(*result)
    return out

In [ ]:
from SCFInitialGuess.utilities.analysis import mf_initializer_damping, measure_iterations, statistics
from SCFInitialGuess.utilities.dataset import make_matrix_batch

print("--- Iterations Damped ---\n" + \
format_results(statistics(list(measure_iterations(
    mf_initializer_damping,
    make_matrix_batch(p_nn, dim, is_triu=True).astype('float64'),
    molecules[ind:]
)))))

In [ ]:
from SCFInitialGuess.utilities.analysis import mf_initializer_diis, measure_iterations, statistics
from SCFInitialGuess.utilities.dataset import make_matrix_batch

print("--- Iterations Damped ---\n" + \
format_results(statistics(list(measure_iterations(
    mf_initializer_diis,
    make_matrix_batch(p_nn, dim, is_triu=True).astype('float64'),
    molecules[ind:]
)))))